In [26]:
import pandas as pd
import altair as alt
import networkx as nx
import nx_altair as nxa
from pydracor import *

dracor = DraCor()

In [27]:
plays = dracor.play_id_to_play_title()
available_german_plays = filtered_grades = dict(filter(lambda play: 'ger0' in play[0], plays.items()))
available_german_plays

{'ger000639': 'Amalie und Charlotte Baronessen von Habenichts: oder die ungleichen Schwestern',
 'ger000642': 'Das Ehrenmaal des Patrioten',
 'ger000631': 'Das Lustlager',
 'ger000624': 'Das Urtheil des Paris',
 'ger000588': 'Der deutsche Student',
 'ger000640': 'Die Gespenster',
 'ger000634': 'Emilie oder die Philosophisch-Verliebten',
 'ger000657': 'Die verhandelte Braut',
 'ger000661': 'Die Weiber-List',
 'ger000663': 'Die gedruckten Theater-Regeln, oder: Welcher Stand ist beßer?',
 'ger000667': 'Die Grands von Frankreich in ihrer wahren Gestalt',
 'ger000637': 'Ein April-Scherz',
 'ger000569': 'Am ersten Mai',
 'ger000171': 'Brot!',
 'ger000041': 'Im Suff',
 'ger000630': 'Die Verpfändung',
 'ger000525': 'Der Comödienfeind',
 'ger000576': 'Prinz Tu-Ta-Tu',
 'ger000039': "Heimg'funden",
 'ger000129': 'Das vierte Gebot',
 'ger000120': 'Der Gwissenswurm',
 'ger000337': 'Der Pfarrer von Kirchfeld',
 'ger000304': 'Die Kreuzelschreiber',
 'ger000054': 'Der Meineidbauer',
 'ger000407': 'Da

In [150]:
# play = Play('ger000088')
play = Play('ger000597')
graphml = play.relations_graphml()
# networkX doesn't support mix of directed+undirected Graphs & nx_altair's arrows look broken
# workaround: make graph undirected
graphml = graphml.replace('directed="true"', 'directed="false"')

G = nx.parse_graphml(graphml)
G.nodes.data()
#G.edges.data()

NodeDataView({'der_ehrnhold': {'label': 'Der ehrnhold', 'Gender': 'MALE', 'Person group': False}, 'hecastus_der_reich_mann': {'label': 'Hecastus, der reich sterbent mann', 'Gender': 'MALE', 'Person group': False}, 'epicuria_die_fraw': {'label': 'Epicuria, sein gemahel', 'Gender': 'FEMALE', 'Person group': False}, 'panocitus_der_ander_knecht': {'label': 'Panocitus, der ander Knecht', 'Gender': 'MALE', 'Person group': False}, 'datrus_der_koch': {'label': 'Datrus, der koch', 'Gender': 'MALE', 'Person group': False}, 'demones_der_erst_freund': {'label': 'Demones, der erst freund', 'Gender': 'MALE', 'Person group': False}, 'economus_der_hausvogt': {'label': 'Economus, der hausvogt', 'Gender': 'MALE', 'Person group': False}, 'philepanis_der_ein_knecht': {'label': 'Philepanis, der ein knecht', 'Gender': 'MALE', 'Person group': False}, 'der_legat': {'label': 'Nomodydastalus, der legat', 'Gender': 'MALE', 'Person group': False}, 'ancilla_die_magd': {'label': 'Ancilla, die magd', 'Gender': 'FEMA

In [152]:
# define the graph layout
layout = nx.shell_layout(G)

# draw graph with nx_altair
chart = nxa.draw_networkx(
    G,
    pos=layout,
    node_tooltip=['label', 'Gender'],
    node_color='lightgray',
    edge_color='Relation',
    width=2
)

# get the edge layer
edges = chart.layer[0]
# get the node layer
nodes = chart.layer[1]

# define relationship filter
relations = pd.DataFrame({'Relation': ['parent_of', 'lover_of', 'related_with', 'associated_with', 'siblings', 'spouses', 'friends']})
selection = alt.selection_point(fields=['Relation'], toggle="true")
color = alt.condition(selection, alt.Color('Relation:N', legend=None), alt.value('lightgray'))
relation_filter = alt.Chart(relations, title=alt.TitleParams('Filter Relationship', anchor='start')).mark_rect().encode(
    y=alt.Y('Relation', title=''), 
    color=color
).add_params(selection)

# add filter
edges = edges.encode(color='Relation').transform_filter(selection)

# filter nodes by gender
# options = ['MALE', 'FEMALE', 'UNKNOWN']
# labels = [option + ' ' for option in options]
# radio_buttons = alt.binding_radio(
#     options=options + [None],
#     labels=labels + ['ALL'],
#     name='Gender: '
# )
# selection = alt.selection_single(
#     fields=['Gender'],
#     bind=radio_buttons,
# )
# nodes = nodes.encode(
#     fill=alt.Color('Gender:N').scale(domain=options)
# ).add_params(
#     selection
# ).transform_filter(
#     selection
# )

# print the Chart layers
(relation_filter | (edges+nodes)).configure_view(
    strokeWidth=0 # remove border
).configure_axis(
    domainOpacity=0 # remove axis
)

alt.HConcatChart(...)